In [ ]:
import pandas as pd
pd.set_option('display.max_columns',400)
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter('ignore')

In [ ]:
%load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [ ]:
import pandas as pd
pd.set_option('display.max_columns',400)
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder
warnings.simplefilter('ignore')


#original = pd.read_csv('/content/calories.csv')
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
submission = pd.read_csv("/content/sample_submission.csv")


#train = pd.concat([train, original], ignore_index=True)
#train['id'] = range(len(train))
train.drop(['id'],inplace=True, axis=1)


#train.loc[train['Sex'].isna(), 'Sex'] = train.loc[train['Sex'].isna(), 'Gender'].map({'female': 0, 'male': 1})
#train.drop([#'User_ID',
            #'Gender'],inplace=True, axis=1)


#  Feature Engineering

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Yeo-Johnson tüm değerlerde çalışır (pozitif-negatif)
pt = PowerTransformer(method='yeo-johnson')

train['Body_Temp_yeo'] = pt.fit_transform(train[['Body_Temp']])
test['Body_Temp_yeo'] = pt.fit_transform(test[['Body_Temp']])

train['Age_log'] = np.log1p(train['Age'])
test['Age_log'] = np.log1p(test['Age'])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)


features = ['Duration', 'Heart_Rate', 'Body_Temp_yeo','Age_log']
# 1. Train için polynomial fit + dönüşüm
X_poly = poly.fit_transform(train[features])
poly_features = poly.get_feature_names_out(features)
poly_df = pd.DataFrame(X_poly, columns=poly_features, index=train.index)
train = pd.concat([train, poly_df], axis=1)

# 2. Test için sadece transform
X_poly_test = poly.transform(test[features])
poly_df_test = pd.DataFrame(X_poly_test, columns=poly_features, index=test.index)
test = pd.concat([test, poly_df_test], axis=1)

# 3. Aynı sütun isimli (ya da birebir aynı değerli) kolonları silme (opsiyonel)
train = train.loc[:, ~train.columns.duplicated()]
test = test.loc[:, ~test.columns.duplicated()]


In [ ]:
TARGET = 'Calories'
CATS = ['Sex'] #daha sonra duration heart_rate ve body_temp dene
NUMS = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

In [ ]:
train['Sex'] = train.Sex.map({'male':1,'female':0})
test['Sex'] = test.Sex.map({'male':1,'female':0})

In [ ]:
train['Body_Temp_minus_cube'] = (train['Body_Temp'] - 35)**3

**MULTIPLYING**

In [ ]:
def add_feature_cross_terms(df, main_features, numerical_features):
    df_new = df.copy()
    used_pairs = set()

    for f1 in main_features:
        for f2 in numerical_features:
            if f1 != f2:
                pair = frozenset([f1, f2])
                if pair not in used_pairs:
                    col_name = f"{f1}_x_{f2}" if f1 < f2 else f"{f2}_x_{f1}"
                    df_new[col_name] = df_new[f1] * df_new[f2]
                    used_pairs.add(pair)

    return df_new

numerical_features = ['Age',
                     # 'Age_log', 'Height', 'Weight',
                      'Duration', 'Heart_Rate',
                      #'Body_Temp',
                      #'Body_Temp_yeo'
                      'Body_Temp_minus_cube',
                      ]
main_features = ['Duration', 'Heart_Rate',
                 'Body_Temp',
                 #'Duration_sq',
                # 'Round_1','Round_2'
            ]

train = add_feature_cross_terms(train, main_features, numerical_features)
test = add_feature_cross_terms(test, main_features, numerical_features)

**TO CATEGORIC**

In [ ]:
train['Duration_num'] = train['Duration']
train['Heart_Rate_num'] = train['Heart_Rate']
train['Body_Temp_num'] = train['Body_Temp']



cat_cols = ['Duration_num', 'Heart_Rate_num',
            'Body_Temp_num'
           ]

for col in cat_cols:
    train[col] = train[col].astype('category')



test['Duration_num'] = test['Duration']
test['Heart_Rate_num'] = test['Heart_Rate']
test['Body_Temp_num'] = test['Body_Temp']

for col in cat_cols:
    test[col] = test[col].astype('category')


**DOMAIN**

In [ ]:
# Normalize HR
train['HR_reserve'] = train['Heart_Rate'] / (220 - train['Age'])
test['HR_reserve'] = test['Heart_Rate'] / (220 - test['Age'])

**ROUND**

In [ ]:
import pandas as pd
import numpy as np
import itertools

def feature_eng(df):
#    df['Round_1'] = df['Duration'].round() ** 2
  #  df['Round_2'] = df['Heart_Rate'].round() ** 2
   # df['Round_3'] = df['Body_Temp'].round() ** 2
    df['d3'] = df['Duration'] // 3
    df['d2'] = df['Duration'] // 2
    df['d4'] = df['Duration'] // 4

    df['hr3'] = df['Heart_Rate'] // 3
    df['hr2'] = df['Heart_Rate'] // 2
    df['hr4'] = df['Heart_Rate'] // 4

    df['b3'] = df['Body_Temp'] // 3
    df['b2'] = df['Body_Temp'] // 2
    df['b4'] = df['Body_Temp'] // 4

# Örnek olarak 'Duration' değişkeni üzerinden
    df['First_Digit-h']  = df['Heart_Rate'].astype(int).astype(str).str[0].astype(int)
    df['Second_Digit-h'] = df['Heart_Rate'].astype(int).astype(str).str[1].astype(int)
# Örnek olarak 'Duration' değişkeni üzerinden
    df['First_Digit-b']  = df['Body_Temp'].astype(int).astype(str).str[0].astype(int)
    df['Second_Digit-b'] = df['Body_Temp'].astype(int).astype(str).str[1].astype(int)

    df['First_Digit-d']  = df['Duration'].astype(int).astype(str).str[0].astype(int)
    #df['Second_Digit-d'] = df['Duration'].astype(int).astype(str).str[1].astype(int)
    return df

train = feature_eng(train)
test  = feature_eng(test)

**MATH**

In [ ]:
train['Duration_sq'] = train['Duration']**2
test['Duration_sq'] = test['Duration']**2

train['Heart_Rate_sq'] = train['Heart_Rate']**2
test['Heart_Rate_sq'] = test['Heart_Rate']**2

train['Body_Temp_sq'] = train['Body_Temp']**2
test['Body_Temp_sq'] = test['Body_Temp']**2

##  Encode Columns

In [ ]:
#encoded columns ureteceksin iterasyonda var

encode_columns = [
    'Age',
    #'Height',
    #'Weight',
    'Duration',
    'Heart_Rate',
    'Body_Temp'
]

encoded_columns = []

In [ ]:
 ## 2-way
# for i in range(len(encode_columns)):
   #  for j in range(i + 1, len(encode_columns)):
   #      col1 = encode_columns[i]
    #     col2 = encode_columns[j]
    #     name = f"{col1}_{col2}"

     #    train[name] = train[col1].astype(str) + '_' + train[col2].astype(str)
      #   test[name] = test[col1].astype(str) + '_' + test[col2].astype(str)

       #  encoded_columns.append(name)

 # # 3-way
 #for i in range(len(encode_columns)):
  #   for j in range(i + 1, len(encode_columns)):
   #      for k in range(j + 1, len(encode_columns)):
    #         col1 = encode_columns[i]
     #        col2 = encode_columns[j]
      #       col3 = encode_columns[k]
       #      name = f"{col1}_{col2}_{col3}"
##
  #           train[name] = train[col1].astype(str) + '_' + train[col2].astype(str) + '_' + train[col3].astype(str)
   #          test[name] = test[col1].astype(str) + '_' + test[col2].astype(str) + '_' + test[col3].astype(str)
##
  #           encoded_columns.append(name)

# 4-way
 #for i in range(len(encode_columns)):
   # for j in range(i + 1, len(encode_columns)):
    #    for k in range(j + 1, len(encode_columns)):
      #      for l in range(k + 1, len(encode_columns)):
       #         col1 = encode_columns[i]
        #        col2 = encode_columns[j]
         #       col3 = encode_columns[k]
          #      col4 = encode_columns[l]
           #     name = f"{col1}_{col2}_{col3}_{col4}"
#
 #               train[name] = (
  #                  train[col1].astype(str) + '_' +
   #                 train[col2].astype(str) + '_' +
    #                train[col3].astype(str) + '_' +
     #               train[col4].astype(str)
      #          )
       #         test[name] = (
        #            test[col1].astype(str) + '_' +
         #           test[col2].astype(str) + '_' +
          #          test[col3].astype(str) + '_' +
           #         test[col4].astype(str)
            #    )
##
  #              encoded_columns.append(name)
#

train[encoded_columns] = train[encoded_columns].astype('category')
test[encoded_columns] = test[encoded_columns].astype('category')
print('Created Interaction Features:', encoded_columns)

Created Interaction Features: []


# FORWARD FEATURE SELECTION USING TARGET ENCODING

In [ ]:
from sklearn.model_selection import KFold
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
import gc

In [ ]:
FEATURES = [col for col in train.columns if col not in [TARGET]]

In [ ]:
FEATURES

['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'Sex']

In [ ]:
FEATURES = NUMS + CATS #+ encoded_columns

In [ ]:
encoded_columns = [col for col in train.columns if col not in ['id', 'Calories']]

In [ ]:
train.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Body_Temp_yeo,Age_log,Age_x_Duration,Age_log_x_Duration,Duration_x_Height,Duration_x_Weight,Duration_x_Heart_Rate,Body_Temp_x_Duration,Body_Temp_yeo_x_Duration,Age_x_Heart_Rate,Age_log_x_Heart_Rate,Heart_Rate_x_Height,Heart_Rate_x_Weight,Body_Temp_x_Heart_Rate,Body_Temp_yeo_x_Heart_Rate,Age_x_Body_Temp,Age_log_x_Body_Temp,Body_Temp_x_Height,Body_Temp_x_Weight,Body_Temp_x_Body_Temp_yeo,Duration_num,Heart_Rate_num,Body_Temp_num,HR_reserve,d3,d2,d4,hr3,hr2,hr4,b3,b2,b4,First_Digit-h,Second_Digit-h,First_Digit-b,Second_Digit-b,First_Digit-d,Duration_sq,Heart_Rate_sq,Body_Temp_sq
0,1,36,189.0,82.0,26.0,101.0,41.0,150.0,1.639052,3.610918,936.0,93.883866,4914.0,2132.0,2626.0,1066.0,42.615351,3636.0,364.702709,19089.0,8282.0,4141.0,165.544246,1476.0,148.047634,7749.0,3362.0,67.201130,26.0,101.0,41.0,0.548913,8.0,13.0,6.0,33.0,50.0,25.0,13.0,20.0,10.0,1,0,4,1,2,676.0,10201.0,1681.00
1,0,64,163.0,60.0,8.0,85.0,39.7,34.0,-0.673548,4.174387,512.0,33.395098,1304.0,480.0,680.0,317.6,-5.388382,5440.0,354.822918,13855.0,5100.0,3374.5,-57.251559,2540.8,165.723175,6471.1,2382.0,-26.739846,8.0,85.0,39.7,0.544872,2.0,4.0,2.0,28.0,42.0,21.0,13.0,19.0,9.0,8,5,3,9,8,64.0,7225.0,1576.09
2,0,51,161.0,64.0,7.0,84.0,39.8,29.0,-0.552252,3.951244,357.0,27.658706,1127.0,448.0,588.0,278.6,-3.865766,4284.0,331.904472,13524.0,5376.0,3343.2,-46.389188,2029.8,157.259500,6407.8,2547.2,-21.979639,7.0,84.0,39.8,0.497041,2.0,3.0,1.0,28.0,42.0,21.0,13.0,19.0,9.0,8,4,3,9,7,49.0,7056.0,1584.04
3,1,20,192.0,90.0,25.0,105.0,40.7,140.0,0.937853,3.044522,500.0,76.113061,4800.0,2250.0,2625.0,1017.5,23.446331,2100.0,319.674856,20160.0,9450.0,4273.5,98.474590,814.0,123.912063,7814.4,3663.0,38.170627,25.0,105.0,40.7,0.525000,8.0,12.0,6.0,35.0,52.0,26.0,13.0,20.0,10.0,1,0,4,0,2,625.0,11025.0,1656.49
4,0,38,166.0,61.0,25.0,102.0,40.6,146.0,0.730318,3.663562,950.0,91.589041,4150.0,1525.0,2550.0,1015.0,18.257941,3876.0,373.683288,16932.0,6222.0,4141.2,74.492399,1542.8,148.740603,6739.6,2476.6,29.650896,25.0,102.0,40.6,0.560440,8.0,12.0,6.0,34.0,51.0,25.0,13.0,20.0,10.0,1,0,4,0,2,625.0,10404.0,1648.36


In [ ]:
#Asagida nested icerisine mean den baska stats eklemek istersen donguyu guncellemen lazim

def target_encode(df_train, df_val, col, target, stats='mean', prefix='TE'):
    agg = df_train.groupby(col)[target].agg(stats).rename(f"{prefix}_{col}")
    df_val = df_val.copy()
    df_val[f"{prefix}_{col}"] = df_val[col].map(agg).astype(float)
    df_val[f"{prefix}_{col}"].fillna(agg.mean(), inplace=True)
    return df_val

In [ ]:
encoded_columns = [col for col in train.columns if col not in NUMS + CATS + ['id', 'Calories']]
FEATURES = NUMS + CATS + encoded_columns
BASE_FEATURES = NUMS + CATS

In [ ]:
# Forward Feature Selection using Target Encoding
ADD = []
best_rmsle = np.inf
best_oof = None
best_pred = None

BASE_FEATURES = [col for col in FEATURES if col not in encoded_columns]

FOLDS = 5
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

for sel_col in ['baseline'] + encoded_columns:
                                #encoded_columns:
    if sel_col != 'baseline':
        ADD.append(sel_col)

    oof = np.zeros((len(train)))
    pred = np.zeros((len(test)))

    for i, (train_idx, valid_idx) in enumerate(kf.split(train)):
        print(f"--- Fold {i+1} / {FOLDS} ---")

        X_train = train.loc[train_idx, BASE_FEATURES + ADD + [TARGET]].reset_index(drop=True)
        y_train = X_train[TARGET]
        X_valid = train.loc[valid_idx, BASE_FEATURES + ADD].reset_index(drop=True)
        y_valid = train.loc[valid_idx, TARGET].reset_index(drop=True)
        X_test = test[BASE_FEATURES + ADD].reset_index(drop=True)

        y_train = np.log1p(y_train)
        y_valid = np.log1p(y_valid)
        trainn =  np.log1p(train[TARGET])
        # --- NESTED K-FOLD TARGET ENCODING ONLY FOR ADD COLUMNS ---
        kf2 = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
        for j, (train_idx2, valid_idx2) in enumerate(kf2.split(X_train)):
            inner_train = X_train.loc[train_idx2, BASE_FEATURES + ADD + [TARGET]].copy()
            inner_valid = X_train.loc[valid_idx2, BASE_FEATURES + ADD].copy()

            for col in ADD:
                inner_valid = target_encode(inner_train, inner_valid, col, TARGET, stats="mean", prefix="TE")

            new_cols = inner_valid.columns.difference(BASE_FEATURES)
            X_train.loc[valid_idx2, new_cols] = inner_valid[new_cols].values

            del inner_train, inner_valid
            gc.collect()

        for col in ADD:
            X_valid = target_encode(X_train, X_valid, col, TARGET, stats="mean", prefix="TE")
            X_test = target_encode(X_train, X_test, col, TARGET, stats="mean", prefix="TE")

        X_train.drop([TARGET] + ADD, axis=1, inplace=True)
        X_valid.drop(ADD, axis=1, inplace=True)
        X_test.drop(ADD, axis=1, inplace=True)

        final_features = [col for col in X_train.columns]
        X_train = X_train[final_features]
        X_valid = X_valid[final_features]
        X_test = X_test[final_features]

        model = XGBRegressor(
            #tree_method='gpu_hist',
            device = 'cuda',
            max_depth=10,
            colsample_bytree=0.6,
            subsample=0.8,
            n_estimators=1000,
            learning_rate=0.1,
            enable_categorical=True,
            early_stopping_rounds=100,
            min_child_weight=10,
        )
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=500)

        oof[valid_idx] = model.predict(X_valid)
        pred += model.predict(X_test)

        del X_train, X_valid, X_test, y_train, y_valid
        if i != FOLDS - 1:
            del model
        gc.collect()


    rmsle_score = np.sqrt(
        mean_squared_log_error(
            np.expm1(trainn),
            np.expm1(oof)
            )
        )

    pred /= FOLDS
    #rmsle_score = mean_squared_error(train[TARGET], oof)

    if rmsle_score < best_rmsle:
        print(f"NEW BEST with {sel_col} at {rmsle_score:.5f}")
        best_rmsle = rmsle_score
        best_oof = oof.copy()
        best_pred = pred.copy()
    else:
        print(f"Worse with {sel_col} at {rmsle_score:.5f}")
        ADD.remove(sel_col)

--- Fold 1 / 5 ---
[0]	validation_0-rmse:0.86988
[294]	validation_0-rmse:0.06028
--- Fold 2 / 5 ---
[0]	validation_0-rmse:0.87206
[345]	validation_0-rmse:0.06127
--- Fold 3 / 5 ---
[0]	validation_0-rmse:0.87091
[298]	validation_0-rmse:0.06050
--- Fold 4 / 5 ---
[0]	validation_0-rmse:0.86951
[306]	validation_0-rmse:0.06090
--- Fold 5 / 5 ---
[0]	validation_0-rmse:0.87032
[279]	validation_0-rmse:0.06023
NEW BEST with baseline at 0.06046
--- Fold 1 / 5 ---
[0]	validation_0-rmse:0.86988
[296]	validation_0-rmse:0.06071
--- Fold 2 / 5 ---
[0]	validation_0-rmse:0.87206
[338]	validation_0-rmse:0.06121
--- Fold 3 / 5 ---
[0]	validation_0-rmse:0.87091
[295]	validation_0-rmse:0.06093
--- Fold 4 / 5 ---
[0]	validation_0-rmse:0.86951
[291]	validation_0-rmse:0.06103
--- Fold 5 / 5 ---
[0]	validation_0-rmse:0.87032
[338]	validation_0-rmse:0.06056
Worse with Body_Temp_yeo at 0.06077
--- Fold 1 / 5 ---
[0]	validation_0-rmse:0.86988
[339]	validation_0-rmse:0.06059
--- Fold 2 / 5 ---
[0]	validation_0-rms

In [ ]:
print( ADD )

[]
